# Multi-Agent 시스템

**Multi-agent 시스템**은 복잡한 애플리케이션을 여러 전문화된 에이전트로 나누어 함께 문제를 해결합니다.

단일 에이전트가 모든 단계를 처리하는 대신, **multi-agent 아키텍처**를 사용하면 더 작고 집중된 에이전트를 조정된 워크플로우로 구성할 수 있습니다.

## Multi-agent가 유용한 경우

- 단일 에이전트가 너무 많은 도구를 가지고 있어 어떤 것을 사용할지 잘못 결정하는 경우
- 컨텍스트 또는 메모리가 한 에이전트가 효과적으로 추적하기에 너무 큰 경우
- 작업에 **전문화**가 필요한 경우 (예: 계획자, 연구자, 수학 전문가)

## Multi-agent 패턴

| 패턴 | 작동 방식 | 제어 흐름 | 사용 사례 |
|------|----------|----------|----------|
| **Tool Calling** | **수퍼바이저** 에이전트가 다른 에이전트를 *도구*로 호출. 도구 에이전트는 사용자와 직접 대화하지 않고 작업을 실행하고 결과를 반환 | 중앙 집중식: 모든 라우팅이 호출 에이전트를 통과 | 작업 오케스트레이션, 구조화된 워크플로우 |
| **Handoffs** | 현재 에이전트가 다른 에이전트로 **제어를 전달**. 활성 에이전트가 변경되고 사용자는 새 에이전트와 직접 상호작용 | 분산형: 에이전트가 활성 에이전트를 변경 가능 | 다중 도메인 대화, 전문가 인계 |

## 패턴 선택 가이드

| 질문 | Tool Calling | Handoffs |
|------|-------------|----------|
| 워크플로우에 대한 중앙 집중식 제어가 필요한가? | ✅ Yes | ❌ No |
| 에이전트가 사용자와 직접 상호작용하기를 원하는가? | ❌ No | ✅ Yes |
| 전문가 간 복잡하고 인간과 같은 대화? | ❌ 제한적 | ✅ 강력함 |

**팁**: 두 패턴을 혼합할 수 있습니다 - 에이전트 전환에는 **handoffs**를 사용하고, 각 에이전트가 전문 작업을 위해 **하위 에이전트를 도구로 호출**하도록 할 수 있습니다.

## 사전 준비

환경 변수를 설정합니다.

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

## Tool Calling 패턴

**Tool calling**에서는 하나의 에이전트("**컨트롤러**")가 다른 에이전트를 필요할 때 호출할 *도구*로 취급합니다.

컨트롤러는 오케스트레이션을 관리하고, 도구 에이전트는 특정 작업을 수행하고 결과를 반환합니다.

### 기본 구현

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini")

# 수학 전문 하위 에이전트
@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions."""
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

math_agent = create_agent(
    model=model,
    tools=[calculator],
    system_prompt="You are a math expert. Solve mathematical problems accurately."
)

# 메인 에이전트를 위한 도구로 수학 에이전트 래핑
@tool(
    "math_expert",
    description="Use this tool for mathematical calculations and problem solving"
)
def call_math_agent(query: str) -> str:
    """Call the math expert agent."""
    result = math_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return result["messages"][-1].content

# 메인 에이전트 (컨트롤러)
main_agent = create_agent(
    model=model,
    tools=[call_math_agent],
    system_prompt="You are a helpful assistant. When users ask math questions, use the math_expert tool."
)

# 테스트
result = main_agent.invoke({
    "messages": [{"role": "user", "content": "What is 15 * 23 + 47?"}]
})

print(result["messages"][-1].content)

### 다중 하위 에이전트

여러 전문 에이전트를 도구로 사용하는 예제입니다.

In [ ]:
# 연구 전문 하위 에이전트
@tool
def search_web(query: str) -> str:
    """Search the web for information."""
    # 실제로는 웹 검색 API 호출
    return f"Search results for: {query}"

research_agent = create_agent(
    model=model,
    tools=[search_web],
    system_prompt="You are a research expert. Find and summarize information on topics."
)

@tool(
    "researcher",
    description="Use this tool to research topics and gather information"
)
def call_research_agent(query: str) -> str:
    """Call the research agent."""
    result = research_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return result["messages"][-1].content

# 글쓰기 전문 하위 에이전트
writing_agent = create_agent(
    model=model,
    tools=[],
    system_prompt="You are a professional writer. Create well-structured, engaging content."
)

@tool(
    "writer",
    description="Use this tool to write articles, reports, or other documents"
)
def call_writing_agent(query: str) -> str:
    """Call the writing agent."""
    result = writing_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    return result["messages"][-1].content

# 수퍼바이저 에이전트
supervisor = create_agent(
    model=model,
    tools=[call_math_agent, call_research_agent, call_writing_agent],
    system_prompt="""You are a supervisor agent that coordinates specialized agents.
    - Use math_expert for calculations
    - Use researcher for information gathering
    - Use writer for content creation
    """
)

print("Supervisor agent with 3 sub-agents created")

### 수퍼바이저 에이전트 테스트

In [ ]:
# 수학 작업
result = supervisor.invoke({
    "messages": [{"role": "user", "content": "Calculate the square root of 144"}]
})
print("Math task:")
print(result["messages"][-1].content)

# 연구 작업
result = supervisor.invoke({
    "messages": [{"role": "user", "content": "Research the history of Python programming language"}]
})
print("\nResearch task:")
print(result["messages"][-1].content)

# 글쓰기 작업
result = supervisor.invoke({
    "messages": [{"role": "user", "content": "Write a short introduction about AI agents"}]
})
print("\nWriting task:")
print(result["messages"][-1].content)

## 컨텍스트 커스터마이징

Multi-agent 설계의 핵심은 **컨텍스트 엔지니어링** - 각 에이전트가 보는 정보를 결정하는 것입니다.

### 하위 에이전트 입력 제어

In [ ]:
from langchain.agents import AgentState
from langchain.tools import tool, ToolRuntime

class CustomState(AgentState):
    user_context: str
    task_history: list

# 컨텍스트를 고려하는 하위 에이전트
context_aware_agent = create_agent(
    model=model,
    tools=[],
    system_prompt="You are a context-aware assistant.",
    state_schema=CustomState
)

@tool(
    "context_aware_tool",
    description="A tool that uses context from the state"
)
def call_context_aware_agent(
    query: str,
    runtime: ToolRuntime[None, CustomState]
) -> str:
    """Call agent with additional context from state."""
    # State에서 컨텍스트 가져오기
    user_context = runtime.state.get("user_context", "")
    task_history = runtime.state.get("task_history", [])

    # 컨텍스트를 포함한 입력 구성
    enhanced_query = f"""
    User Context: {user_context}
    Previous Tasks: {task_history}

    Current Query: {query}
    """

    result = context_aware_agent.invoke({
        "messages": [{"role": "user", "content": enhanced_query}],
        "user_context": user_context,
        "task_history": task_history
    })

    return result["messages"][-1].content

# 컨텍스트를 사용하는 메인 에이전트
main_with_context = create_agent(
    model=model,
    tools=[call_context_aware_agent],
    state_schema=CustomState
)

# 테스트
result = main_with_context.invoke({
    "messages": [{"role": "user", "content": "Help me with this task"}],
    "user_context": "Premium user, prefers detailed explanations",
    "task_history": ["task1", "task2"]
})

print(result["messages"][-1].content)

### 하위 에이전트 출력 제어

In [ ]:
from typing import Annotated
from langchain.tools import InjectedToolCallId
from langchain_core.messages import ToolMessage
from langgraph.types import Command

class OutputState(AgentState):
    analysis_metadata: dict

# 메타데이터를 생성하는 분석 에이전트
analysis_agent = create_agent(
    model=model,
    tools=[],
    system_prompt="Analyze the query and provide insights with metadata.",
    state_schema=OutputState
)

@tool(
    "analyzer",
    description="Analyze data and return results with metadata"
)
def call_analyzer(
    query: str,
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Call analyzer and return results with metadata."""
    result = analysis_agent.invoke({
        "messages": [{"role": "user", "content": query}],
        "analysis_metadata": {}
    })

    # 결과와 함께 메타데이터 반환
    return Command(update={
        "analysis_metadata": {
            "query_length": len(query),
            "timestamp": "2024-01-01",
            "agent": "analyzer"
        },
        "messages": [
            ToolMessage(
                content=result["messages"][-1].content,
                tool_call_id=tool_call_id
            )
        ]
    })

# 메타데이터를 활용하는 메인 에이전트
main_with_metadata = create_agent(
    model=model,
    tools=[call_analyzer],
    state_schema=OutputState
)

result = main_with_metadata.invoke({
    "messages": [{"role": "user", "content": "Analyze this data"}],
    "analysis_metadata": {}
})

print("Result:", result["messages"][-1].content)
print("\nMetadata:", result.get("analysis_metadata", {}))

## 실용적인 예제: 고객 지원 시스템

여러 전문 에이전트로 구성된 고객 지원 시스템을 만들어봅시다.

In [ ]:
# 기술 지원 에이전트
@tool
def check_system_status(system: str) -> str:
    """Check the status of a system."""
    return f"System {system} is operational"

@tool
def restart_service(service: str) -> str:
    """Restart a service."""
    return f"Service {service} restarted successfully"

tech_support = create_agent(
    model=model,
    tools=[check_system_status, restart_service],
    system_prompt="""You are a technical support specialist.
    Help users troubleshoot technical issues.
    Always check system status first before suggesting solutions."""
)

@tool(
    "tech_support",
    description="Handle technical support issues, troubleshooting, and system problems"
)
def call_tech_support(issue: str) -> str:
    result = tech_support.invoke({
        "messages": [{"role": "user", "content": issue}]
    })
    return result["messages"][-1].content

# 청구 지원 에이전트
@tool
def check_invoice(invoice_id: str) -> str:
    """Check invoice details."""
    return f"Invoice {invoice_id}: Amount $100, Status: Paid"

@tool
def process_refund(order_id: str) -> str:
    """Process a refund."""
    return f"Refund processed for order {order_id}"

billing_support = create_agent(
    model=model,
    tools=[check_invoice, process_refund],
    system_prompt="""You are a billing support specialist.
    Help users with invoices, payments, and refunds.
    Always verify invoice details before processing refunds."""
)

@tool(
    "billing_support",
    description="Handle billing questions, invoices, payments, and refunds"
)
def call_billing_support(issue: str) -> str:
    result = billing_support.invoke({
        "messages": [{"role": "user", "content": issue}]
    })
    return result["messages"][-1].content

# 일반 문의 에이전트
general_support = create_agent(
    model=model,
    tools=[],
    system_prompt="""You are a general customer support agent.
    Answer general questions about products and services.
    Be friendly and helpful."""
)

@tool(
    "general_support",
    description="Handle general inquiries about products, services, and company information"
)
def call_general_support(question: str) -> str:
    result = general_support.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    return result["messages"][-1].content

# 수퍼바이저 고객 지원 에이전트
customer_support = create_agent(
    model=model,
    tools=[call_tech_support, call_billing_support, call_general_support],
    system_prompt="""You are a customer support supervisor.
    Route customer inquiries to the appropriate specialist:
    - tech_support: Technical issues, errors, system problems
    - billing_support: Invoices, payments, refunds
    - general_support: General questions, product information

    Determine which specialist to call based on the customer's issue."""
)

print("Customer support system created")

### 고객 지원 시스템 테스트

In [ ]:
# 기술 문제
print("=== Technical Issue ===")
result = customer_support.invoke({
    "messages": [{"role": "user", "content": "My application keeps crashing. Can you help?"}]
})
print(result["messages"][-1].content)

# 청구 문제
print("\n=== Billing Issue ===")
result = customer_support.invoke({
    "messages": [{"role": "user", "content": "I need a refund for order #12345"}]
})
print(result["messages"][-1].content)

# 일반 문의
print("\n=== General Inquiry ===")
result = customer_support.invoke({
    "messages": [{"role": "user", "content": "What are your business hours?"}]
})
print(result["messages"][-1].content)

## 고급 패턴: 계층적 에이전트

에이전트가 여러 레벨의 계층 구조를 가질 수 있습니다.

In [ ]:
# Level 3: 기본 작업 에이전트
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

basic_math = create_agent(
    model=model,
    tools=[add_numbers],
    system_prompt="You perform basic arithmetic operations."
)

@tool("basic_math", description="Perform basic arithmetic")
def call_basic_math(query: str) -> str:
    result = basic_math.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

# Level 2: 중간 수준 에이전트
intermediate_math = create_agent(
    model=model,
    tools=[call_basic_math],
    system_prompt="You solve intermediate math problems using basic operations."
)

@tool("intermediate_math", description="Solve intermediate math problems")
def call_intermediate_math(query: str) -> str:
    result = intermediate_math.invoke({"messages": [{"role": "user", "content": query}]})
    return result["messages"][-1].content

# Level 1: 최상위 에이전트
advanced_math = create_agent(
    model=model,
    tools=[call_intermediate_math],
    system_prompt="You solve complex math problems by breaking them into simpler parts."
)

# 테스트
result = advanced_math.invoke({
    "messages": [{"role": "user", "content": "Calculate (5 + 3) + (10 + 7)"}]
})

print("Hierarchical result:", result["messages"][-1].content)

## 모범 사례

### 1. 명확한 도구 설명

하위 에이전트의 이름과 설명을 명확하게 작성하세요.

In [ ]:
# 좋은 예
@tool(
    "sql_expert",
    description="""Use this for database queries and SQL-related tasks.
    Capabilities:
    - Write SELECT, INSERT, UPDATE queries
    - Optimize database performance
    - Explain query plans

    Do NOT use for:
    - General programming questions
    - Data analysis (use data_analyst instead)"""
)
def call_sql_expert(query: str) -> str:
    pass

# 나쁜 예
@tool(
    "helper",
    description="Helps with stuff"
)
def call_helper(query: str) -> str:
    pass

### 2. 적절한 컨텍스트 전달

하위 에이전트에 필요한 컨텍스트만 전달하세요.

In [ ]:
# 좋은 예: 필요한 컨텍스트만 전달
@tool
def call_subagent(query: str, runtime: ToolRuntime) -> str:
    """
    call this agent with user_id and task_type
    """
    relevant_context = {
        "user_id": runtime.state.get("user_id"),
        "task_type": runtime.state.get("task_type")
    }
    # 필요한 컨텍스트만 전달
    return "result"

# 나쁜 예: 모든 상태 전달
@tool
def call_subagent_bad(query: str, runtime: ToolRuntime) -> str:
    # 전체 상태를 전달하면 불필요한 정보가 포함됨
    entire_state = runtime.state
    return "result"

### 3. 결과 포맷 표준화

하위 에이전트의 출력 형식을 일관되게 유지하세요.

In [ ]:
# 표준 응답 형식
def standardize_response(agent_result: dict) -> str:
    """Standardize agent response format."""
    content = agent_result["messages"][-1].content

    # 추가 메타데이터 포함
    return f"[Agent Response]\n{content}\n[End Response]"

@tool
def call_standardized_agent(query: str) -> str:
    result = some_agent.invoke({"messages": [{"role": "user", "content": query}]})
    return standardize_response(result)

### 4. 에러 처리

하위 에이전트 호출 시 적절한 에러 처리를 구현하세요.

In [ ]:
@tool
def call_agent_with_error_handling(query: str) -> str:
    """Call agent with proper error handling."""
    try:
        result = some_agent.invoke({
            "messages": [{"role": "user", "content": query}]
        })
        return result["messages"][-1].content
    except Exception as e:
        # 에러를 명확히 보고
        return f"Error calling agent: {str(e)}. Please try rephrasing your request."

### 5. 성능 모니터링

각 에이전트의 성능을 추적하세요.

In [ ]:
import time

@tool
def call_monitored_agent(query: str) -> str:
    """Call agent with performance monitoring."""
    start_time = time.time()

    result = some_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })

    duration = time.time() - start_time
    print(f"Agent execution time: {duration:.2f}s")

    return result["messages"][-1].content

## 요약

Multi-agent 시스템을 사용하면:

1. **전문화**: 각 에이전트가 특정 작업에 집중
2. **확장성**: 새로운 에이전트를 쉽게 추가
3. **유지보수성**: 독립적인 에이전트로 관리 용이
4. **신뢰성**: 전문 에이전트가 더 나은 결정을 내림

### Tool Calling 패턴

- 중앙 집중식 제어
- 구조화된 워크플로우
- 하위 에이전트는 도구로 작동

### Handoffs 패턴

- 분산형 제어
- 에이전트 간 직접 전환
- 사용자와의 직접 상호작용 (구현 예정)

적절한 패턴을 선택하고 컨텍스트를 잘 엔지니어링하면 강력하고 확장 가능한 AI 시스템을 구축할 수 있습니다.